***Adding borders to census tract data***

In [1]:
# import Exports/Data/CensusTractExposure.csv
import pandas as pd
import geopandas as gpd

filepath = 'Exports/Data/7.CensusTractExposure.csv'
exposure_data_2010 = pd.read_csv(filepath)

tract_path_2010 = "Imports/CensusTractGIS/2010/US_tract_2010.shp"
gdf_2010 = gpd.read_file(tract_path_2010)

In [2]:
exposure_data_2010 = pd.merge(exposure_data_2010, gdf_2010[['GISJOIN', 'geometry']], on=['GISJOIN'], how='left', indicator=True, validate='1:1')
exposure_data_2010['_merge'].value_counts()

_merge
both          73669
left_only       333
right_only        0
Name: count, dtype: int64

In [3]:
print('Tracts with no border:', exposure_data_2010.query("_merge == 'left_only'")['TRACTCE'].unique())

Tracts with no border: [990000 990100 990200 990300 990400 990500 990600 990700 990800 990900
 991000 991100 991200 991300 991400 991500 991600 991700 990003 990001
 990301 990302   8900 990101 992201 990002 992900 990201 990501 990021
 990103 992600 990016 991800 990025 990013 993000 992700 992800]


In [4]:
print('dropping mismatches')
exposure_data_2010 = exposure_data_2010.query("_merge == 'both'")
exposure_data_2010 = exposure_data_2010.drop(columns=['_merge'])

dropping mismatches


In [5]:
exposure_data_2010 = gpd.GeoDataFrame(exposure_data_2010, geometry='geometry')

In [6]:
print(exposure_data_2010.crs)
exposure_data_2010 = exposure_data_2010.to_crs(epsg=4326)
exposure_data_2010['geometry'] = exposure_data_2010['geometry'].simplify(tolerance=0.001)
print(exposure_data_2010.crs)

PROJCS["USA_Contiguous_Albers_Equal_Area_Conic",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",37.5],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","102003"]]
EPSG:4326


In [7]:
exposure_data_2010.head()

,STATEFP,COUNTYFP,TRACTCE,state_abbr,county_name,weighted_ave_HH_conc_Hip Hop,weighted_ave_HH_conc_Black,weighted_ave_HH_conc_Urban Contemporary,weighted_ave_HH_conc_Hip Hop_Black,weighted_ave_HH_conc_Hip Hop_Urban Contemporary,...,non_weighted_sum_HH_conc_Urban Contemporary,non_weighted_sum_HH_conc_Hip Hop_Black,non_weighted_sum_HH_conc_Hip Hop_Urban Contemporary,non_weighted_sum_HH_conc_Black_Urban Contemporary,non_weighted_sum_HH_conc_Hip Hop_Black_Urban Contemporary,initial_exp_Hip Hop,initial_exp_Black,initial_exp_Urban Contemporary,GISJOIN,geometry
0,1,1,20100,AL,Autauga County,NaN,NaN,NaN,NaN,NaN,...,0.2222,0.0,0.2222,0.2222,0.2222,NaN,1974.0,1991.0,G0100010020100,"POLYGON ((-86.47886 32.48934, -86.48122 32.476..."
1,1,1,20200,AL,Autauga County,NaN,NaN,NaN,NaN,NaN,...,0.2222,0.0,0.2222,0.2222,0.2222,NaN,1974.0,1991.0,G0100010020200,"POLYGON ((-86.46754 32.46696, -86.46764 32.456..."
2,1,1,20300,AL,Autauga County,NaN,NaN,NaN,NaN,NaN,...,0.2222,0.0,0.2222,0.2222,0.2222,NaN,1974.0,1991.0,G0100010020300,"POLYGON ((-86.45068 32.46034, -86.46735 32.459..."
3,1,1,20400,AL,Autauga County,NaN,NaN,NaN,NaN,NaN,...,0.2222,0.0,0.2222,0.2222,0.2222,NaN,1974.0,1991.0,G0100010020400,"POLYGON ((-86.43643 32.48299, -86.4336 32.4706..."
4,1,1,20500,AL,Autauga County,NaN,NaN,NaN,NaN,NaN,...,0.2222,0.0,0.2222,0.2222,0.2222,NaN,1974.0,1991.0,G0100010020500,"POLYGON ((-86.41135 32.42399, -86.41622 32.430..."


In [8]:
exposure_data_2010.to_file('Exports/Data/8.CensusTractExposure.geojson', driver='GeoJSON')

***Doing the same for counties***

In [9]:
county_path_2010 = "Imports/CountyGIS/US_county_2010.shp"
exposure_path_2010 = "Exports/Data/7.CountyExposure.csv"
gdf_2010 = gpd.read_file(county_path_2010)
exposure_data_2010 = pd.read_csv(exposure_path_2010)

In [10]:
gdf_2010.rename(columns={'STATEFP10': 'STATEFP', 'COUNTYFP10': 'COUNTYFP'}, inplace=True)
exposure_data_2010['STATEFP'] = exposure_data_2010['STATEFP'].astype(str)
exposure_data_2010['COUNTYFP'] = exposure_data_2010['COUNTYFP'].astype(str)
exposure_data_2010['GISJOIN'] = 'G' + exposure_data_2010['STATEFP'].str.zfill(2) + exposure_data_2010['COUNTYFP'].str.zfill(4) + '0'

In [11]:
exposure_data_2010 = pd.merge(exposure_data_2010, gdf_2010[['GISJOIN', 'geometry']], on=['GISJOIN'], how='left', indicator=True, validate='1:1')
assert exposure_data_2010['_merge'].unique() == 'both'
exposure_data_2010.drop(columns=['_merge'], inplace=True)

In [12]:
exposure_data_2010 = gpd.GeoDataFrame(exposure_data_2010, geometry='geometry')

In [13]:
print(exposure_data_2010.crs)
exposure_data_2010 = exposure_data_2010.to_crs(epsg=4326)
exposure_data_2010['geometry'] = exposure_data_2010['geometry'].simplify(tolerance=0.001)
print(exposure_data_2010.crs)

PROJCS["USA_Contiguous_Albers_Equal_Area_Conic",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",37.5],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","102003"]]
EPSG:4326


In [14]:
exposure_data_2010.head()

,STATEFP,COUNTYFP,state_abbr,county_name,weighted_ave_HH_conc_Hip Hop,weighted_ave_HH_conc_Black,weighted_ave_HH_conc_Urban Contemporary,weighted_ave_HH_conc_Hip Hop_Black,weighted_ave_HH_conc_Hip Hop_Urban Contemporary,weighted_ave_HH_conc_Black_Urban Contemporary,...,non_weighted_sum_HH_conc_Urban Contemporary,non_weighted_sum_HH_conc_Hip Hop_Black,non_weighted_sum_HH_conc_Hip Hop_Urban Contemporary,non_weighted_sum_HH_conc_Black_Urban Contemporary,non_weighted_sum_HH_conc_Hip Hop_Black_Urban Contemporary,initial_exp_Hip Hop,initial_exp_Black,initial_exp_Urban Contemporary,GISJOIN,geometry
0,1,1,AL,Autauga County,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0030,0.0,0.0030,0.0030,0.0030,NaN,1974.0,1991.0,G0100010,"POLYGON ((-86.41117 32.40993, -86.42507 32.402..."
1,1,3,AL,Baldwin County,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0115,0.0,0.0115,0.0115,0.0115,NaN,1974.0,1987.0,G0100030,"MULTIPOLYGON (((-87.55444 30.27956, -87.55486 ..."
2,1,5,AL,Barbour County,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0004,0.0,0.0004,0.0004,0.0004,NaN,1980.0,1984.0,G0100050,"POLYGON ((-85.05603 32.06305, -85.05883 32.046..."
3,1,7,AL,Bibb County,0.0,0.0,0.3738,0.0,0.335,0.3738,...,0.0009,0.0,0.0009,0.0009,0.0009,NaN,1974.0,1987.0,G0100070,"POLYGON ((-86.89852 33.04981, -86.87687 33.049..."
4,1,9,AL,Blount County,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0010,0.0,0.0010,0.0010,0.0010,NaN,1975.0,1994.0,G0100090,"POLYGON ((-86.30352 34.09907, -86.32266 34.072..."


In [15]:
# Save as geojson
exposure_data_2010.to_file('Exports/Data/8.CountyExposure.geojson', driver='GeoJSON')